In [1]:
cd ../Python/

/home/mfruman/DataSci/PBN/Python


In [2]:
import readPBNData.description as rd
import readPBNData.images as ri

In [3]:
csvs, fileLike = rd.openZip('../Data/train_info.csv.zip')
lines, head = rd.readCSV(fileLike[0])

In [4]:
cols = rd.columns(lines,head)
cols.keys()

['style', 'title', 'artist', 'filename', 'date', 'genre']

In [5]:
# find 10 artists with most paintings in data set
artistTable = rd.table(cols['artist'])
artistTable[:5]

[('3cc9a44380296d93e68b71a27643c25f', 413),
 ('96e7b1bc8d52e18caf0af34fec2e9bcb', 402),
 ('d8a3c897c506be7de91d8f892f14f934', 401),
 ('3f8dc381ccfe9d5cc88b75970262715b', 400),
 ('10bc951c2eb4a2f05fa773bdaace4e3b', 397)]

In [7]:
# images in train_1.zip and train_2.zip file
train_1_names = rd.imagesInZip('../Data/train_1.zip')
train_2_names = rd.imagesInZip('../Data/train_2.zip')

In [8]:
# number of paintings by artist in train_1 and train_2
# distinct artists (slowish)
distinctArtists = [a[0] for a in artistTable]
paintingsIn12 = []
for artist in distinctArtists:
    paintingsIn12.append(len(rd.sameArtist(artist, cols, imageList=train_1_names))
                        +len(rd.sameArtist(artist, cols, imageList=train_2_names)))
artistTable12 = sorted(zip(distinctArtists, paintingsIn12), 
                       key=lambda item: item[1], reverse=True)
artistTable12[:10]

[('0eeac4ecff259dc515be795e1a76019a', 119),
 ('dd4989789d310581024ae2b9203d5439', 111),
 ('121fffad1eb6f7dff228b8a71b6aec72', 105),
 ('1a8d67dbb446bdc4298cc0be56932a38', 104),
 ('ce3d8977aae5986601232aa58d15282a', 103),
 ('c16781c4321948227193214b68477a5c', 101),
 ('3cc9a44380296d93e68b71a27643c25f', 100),
 ('d09f796f2b0aa11dffc88badd9806119', 100),
 ('5aabfc58470d01bb2362795a44a2603b', 99),
 ('db1318d32df7428076e03513ebf762bb', 98)]

In [9]:
# lists of paintings by the 10 "leading" artists in train_1 and train_2 
leaders = [a[0] for a in artistTable12[:10]]
portfolios1 = []
portfolios2 = []
for artist in leaders:
    portfolios1.append(rd.sameArtist(artist, cols, imageList=train_1_names))
    portfolios2.append(rd.sameArtist(artist, cols, imageList=train_2_names))
portfolios = zip(leaders,portfolios1,portfolios2)

In [14]:
# create miniatures of the paintings by leading artists and save them to disk
miniportfolios = []
for portfolio in portfolios:
    minifiles_1 = ri.miniatures('../Data/train_1.zip', \
                                portfolio[1],prefix='train_1',size=(100,100))
    minifiles_2 = ri.miniatures('../Data/train_2.zip', \
                                portfolio[2],prefix='train_2',size=(100,100))
    miniportfolios.append((portfolio[0],minifiles_1,minifiles_2))

In [15]:
# create cutouts of the paintings by leading artists and save them to disk
cutoutportfolios = []
for portfolio in portfolios:
    cutoutfiles_1 = ri.cutouts('../Data/train_1.zip', \
                               portfolio[1],prefix='train_1',size=(100,100))
    cutoutfiles_2 = ri.cutouts('../Data/train_2.zip', \
                               portfolio[2],prefix='train_2',size=(100,100))
    cutoutportfolios.append((portfolio[0],cutoutfiles_1,cutoutfiles_2))

In [12]:
# prepare 1000 pairs of paintings by same artist 
# and 1000 pairs by different artists
import random
npairs = 1000
randomSeed = 666
random.seed(randomSeed)
portfolios_uni = [(artist, p1 + p2) for (artist,p1,p2) in portfolios]
pairs = []
for isample in xrange(npairs):
    # randomly choose an artist
    portfolio = random.choice(portfolios_uni)
    # randomly choose two paintings
    paintings = random.sample(portfolio[1],2)
    pairs.append([paintings[0],paintings[1],1])
for isample in xrange(npairs):
    # randomly choose two artists
    portfoliopair = random.sample(portfolios_uni,2)
    # randomly choose a painting from each artist
    paintings = random.choice(portfoliopair[0][1]), random.choice(portfoliopair[1][1])
    pairs.append([paintings[0],paintings[1],0])

In [13]:
pairs[:5]

[['28512.jpg', '1744.jpg', 1],
 ['26672.jpg', '2113.jpg', 1],
 ['11130.jpg', '26235.jpg', 1],
 ['2238.jpg', '27608.jpg', 1],
 ['10029.jpg', '28898.jpg', 1]]